In [5]:
#imports
import numpy as np

In [26]:
#load small input
input = open("./../../data/day_18/small_input.txt", "r").readlines()

In [38]:
#load input
input = open("./../../data/day_18/input.txt", "r").readlines()

In [39]:
#process input
instructions = []
for line in input:
    line = line.strip()
    d,l,c = line.split()
    instructions.append( (d,int(l),c.replace("(","").replace(")","") ) )

In [17]:
# part 1
trench_positions =  []
trench_colors = []
pos = (0,0)
dir = 3 # 0 = right, 1 = down, 2 = left, 3 = up (global)
trench_positions.append(pos)
for instruction in instructions:
    match instruction[0]:
        case "R": dir = 0
        case "D": dir = 1
        case "L": dir = 2
        case "U": dir = 3
    for i in range(instruction[1]):
        match dir:
            case 0: pos = (pos[0],pos[1]+1)
            case 1: pos = (pos[0]+1,pos[1])
            case 2: pos = (pos[0],pos[1]-1)
            case 3: pos = (pos[0]-1,pos[1])
        trench_positions.append(pos)
        trench_colors.append(instruction[2])

# translate into array
min_x = min([pos[0] for pos in trench_positions])
print(f"min_x: {min_x}")
max_x = max([pos[0] for pos in trench_positions])
print(f"max_x: {max_x}")
min_y = min([pos[1] for pos in trench_positions])
print(f"min_y: {min_y}")
max_y = max([pos[1] for pos in trench_positions])
print(f"max_y: {max_y}")

arr = np.zeros((max_x-min_x+1,max_y-min_y+1),dtype=int)
for pos in trench_positions:
    arr[pos[0]+abs(min_x),pos[1]+abs(min_y)] = 1

#add margin
arr = np.pad(arr,((1,1),(1,1)), 'constant')

min_x: 0
max_x: 9
min_y: 0
max_y: 6


In [18]:
#compute part 1 with bfs
#perform bfs
visited = np.zeros(arr.shape, dtype=bool)
visited[0,0] = True

def get_neighbours(pos,matrix_shape):
    x,y = pos
    xm,ym = matrix_shape
    neighbours = []
    if x > 0:
        neighbours.append((x-1,y))
    if y > 0:
        neighbours.append((x,y-1))
    if x < xm-1:
        neighbours.append((x+1,y))
    if y < ym-1:
        neighbours.append((x,y+1))
    return neighbours

queue = [(0,0)]
while len(queue) > 0:
    x,y = queue.pop(0)
    for nx,ny in get_neighbours((x,y), arr.shape):
        if (not visited[nx,ny]) and (arr[nx,ny] == 0): #for some reason accessing expanded_p here returns an int instead of some string type, therefore the check works with == 0 instead of == '0'
            visited[nx,ny] = True
            queue.append((nx,ny))    
#count tiles
enclosed = 0
for i in range(visited.shape[0]):
    for j in range(visited.shape[1]):
        if not visited[i,j]:
            enclosed += 1

print(f"part 1: {enclosed}")

part 1: 62


In [40]:
#part 2
##switch to shoelace theorem because my computer does not have 1.3tb of RAM :(
trench_positions =  []
trench_colors = []
pos = (0,0)
dir = 3 # 0 = right, 1 = down, 2 = left, 3 = up (global)
trench_positions.append(pos)
perimeter = 0
for instruction in instructions:
    #recalculate instruction from hexadecimal
    length = instruction[2][1:-1]
    length = int(length,16)
    perimeter += length

    dir = int(instruction[2][-1])

    match dir:
        case 0: pos = (pos[0],pos[1]+length)
        case 1: pos = (pos[0]+length,pos[1])
        case 2: pos = (pos[0],pos[1]-length)
        case 3: pos = (pos[0]-length,pos[1])

    trench_positions.append(pos)

# trench positions are now corners which can be used for shoelace theorem

In [42]:
# compute area with shoelace theorem
if trench_positions[0] == trench_positions[-1]:
    trench_positions.pop(-1)
a = 0
for i in reversed(range(len(trench_positions))):
    (x1,y1) = trench_positions[i-1] if i > 0 else trench_positions[-1]
    (x2,y2) = trench_positions[i]
    a += x1*y2-x2*y1
a *= 0.5
a = abs(a)
a += perimeter // 2 + 1

print(f"part 2: {a}")

part 2: 79088855654037.0
